In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Using numpy backend.


Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [2]:
b = np.fromfile('tensors_2010.dat',dtype=np.float64).reshape((8000,-1,200))
Y = np.fromfile('tensors_2010_labels.dat',dtype=np.int)
position_tag_1 = np.fromfile('tensors_2010_entity1.dat',dtype=np.int)
position_tag_2 = np.fromfile('tensors_2010_entity2.dat',dtype=np.int)
position_tag = np.concatenate((position_tag_1,position_tag_2)).reshape((8000,-1))
b.shape,Y.shape,position_tag_1.shape,position_tag_2.shape,position_tag.shape

((8000, 85, 200), (8000,), (8000,), (8000,), (8000, 2))

In [3]:
inputs = []
for i in range(8000):
    inputs.append((torch.from_numpy(b[i]),Y[i]))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [4]:
t_e = []
t_y = []
for E,Y in test:
    t_e.append(E)
    t_y.append(Y)
t_e = torch.stack(t_e)
t_y = torch.from_numpy(np.array(t_y))
t_e.shape, t_y.shape

(torch.Size([2400, 85, 200]), torch.Size([2400]))

# 0.6170212765957447 23轮

In [5]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 53, 200)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=85,    # n_filters
                kernel_size=(1,200),      # filter size
            ),      # output shape (16, 53, 200)
            nn.ReLU(),    # activation
            nn.MaxPool2d(kernel_size=(85,1)),    # 在 2x2 空间里向下采样, output shape (16, 53, 1)
        )
        self.output = nn.Linear(85, 10)   # fully connected layer, output 10 classes

    def forward(self, x):
        torch.set_default_dtype(torch.double)
        x = x.reshape((-1,1,85,200)).double()
        x = self.conv1(x.double())
        x = x.view(x.size(0), -1)   # 展平
#         x = self.out(x)
        output = self.output(x)
        return output

In [7]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = CNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
start = time.time()
for epoch in range(1,301): 
    for step, (batch_x, batch_y) in enumerate(loader):
        pred = net(batch_x)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_e)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss} time: {time.time() - start}")

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 85, kernel_size=(1, 200), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(85, 1), stride=(85, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (output): Linear(in_features=85, out_features=10, bias=True)
)
1: accuracy:0.17916666666666667 loss: 2.146329321607155 time: 1.1540908813476562
2: accuracy:0.525 loss: 1.041457438806205 time: 4.637909889221191
3: accuracy:0.5691666666666667 loss: 0.77110157895037 time: 8.3856680393219
4: accuracy:0.57875 loss: 0.6551115168887782 time: 12.139734983444214
5: accuracy:0.5779166666666666 loss: 0.5794869122309064 time: 16.12717604637146
6: accuracy:0.57 loss: 0.38078476767853314 time: 20.473067045211792
7: accuracy:0.5441666666666667 loss: 0.4189692483672614 time: 25.96641993522644
8: accuracy:0.545 loss: 0.3762888939005211 time: 31.09431791305542
9: accuracy:0.55625 loss: 0.27092003395490966 time: 35.53653812408447
10: accuracy:0.5404166666666667 loss: 0.13612011870426288 time: 40.857999

103: accuracy:0.5383333333333333 loss: 0.0018715206817196302 time: 431.49422788619995
104: accuracy:0.5316666666666666 loss: 0.0025289028217073545 time: 435.77449584007263
105: accuracy:0.5408333333333334 loss: 0.0015372050808740688 time: 439.9542009830475
106: accuracy:0.5425 loss: 0.0007837731214967733 time: 444.0490641593933
107: accuracy:0.5279166666666667 loss: 0.04968174517377747 time: 448.2536370754242
108: accuracy:0.5391666666666667 loss: 0.10821385528241688 time: 452.34173607826233
109: accuracy:0.53875 loss: 0.00737483442413672 time: 456.47475695610046
110: accuracy:0.5341666666666667 loss: 0.0014712077090381368 time: 460.5612678527832
111: accuracy:0.5275 loss: 0.0034949366017675123 time: 464.5616750717163
112: accuracy:0.5254166666666666 loss: 0.00938161999927314 time: 468.7561900615692
113: accuracy:0.5108333333333334 loss: 0.5458309414842806 time: 472.81364917755127
114: accuracy:0.4845833333333333 loss: 0.6062209291178869 time: 476.9093601703644
115: accuracy:0.50416666

205: accuracy:0.53375 loss: 0.0002216413431419921 time: 858.6821300983429
206: accuracy:0.5279166666666667 loss: 0.000314714283563524 time: 862.890340089798
207: accuracy:0.5258333333333334 loss: 0.00025114605736566153 time: 867.026242017746
208: accuracy:0.5333333333333333 loss: 8.072308817017038e-05 time: 871.0710310935974
209: accuracy:0.5329166666666667 loss: 0.0004901917445028388 time: 875.140053987503
210: accuracy:0.5333333333333333 loss: 2.7852002276479437e-05 time: 879.2627260684967
211: accuracy:0.5375 loss: 0.02955060640756856 time: 883.3182001113892
212: accuracy:0.5325 loss: 0.0004897716577101988 time: 887.4345610141754
213: accuracy:0.5266666666666666 loss: 0.13052275345282494 time: 891.5544838905334
214: accuracy:0.53625 loss: 0.0027314240600619915 time: 895.6745059490204
215: accuracy:0.5358333333333334 loss: 0.00016214821318762986 time: 899.9234569072723
216: accuracy:0.5341666666666667 loss: 0.00016163343106925798 time: 904.1719110012054
217: accuracy:0.52916666666666